In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import spacy
import datasets
import torchtext
import tqdm
import evaluate


In [2]:
seed = 1234

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True


## 代码解析

### 1. `os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'` 的作用

**改变了什么**：  
这行代码将Hugging Face的默认API端点从官方源`https://huggingface.co`改为国内镜像源`https://hf-mirror.com`。

**为什么使用镜像源**：
- 🚀 **加速下载**：国内服务器网络延迟更低，下载速度更快
- 🔒 **稳定连接**：避免国际网络波动导致的中断
- 🌐 **绕过限制**：某些网络环境无法直接访问原始域名
- ⚖️ **负载均衡**：减轻官方服务器压力

速度对比：
- 官方源：200KB/s ~ 1MB/s
- 镜像源：通常可达5MB/s ~ 20MB/s

### 2. `dataset = datasets.load_dataset("bentrevett/multi30k")` 的功能

**完成的操作**：
1. **检查缓存**：先查找`~/.cache/huggingface/datasets`目录
2. **下载数据**：若未缓存则从镜像站下载
3. **格式转换**：自动转换为高效的Arrow格式
4. **返回对象**：返回包含三个子集的`DatasetDict`对象

**数据结构**：
```python
{
    'train': Dataset(29000条),     # 训练集
    'validation': Dataset(1014条), # 验证集
    'test': Dataset(1000条)       # 测试集
}

In [3]:
import os
from datasets import load_dataset

# 1. 基础修改方式
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

# 2. 加载数据集（自动走镜像）
dataset = load_dataset("multi30k")


In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['en', 'de'],
        num_rows: 29000
    })
    validation: Dataset({
        features: ['en', 'de'],
        num_rows: 1014
    })
    test: Dataset({
        features: ['en', 'de'],
        num_rows: 1000
    })
})

In [5]:
train_data, valid_data, test_data = (
    dataset["train"],
    dataset["validation"],
    dataset["test"],
)

In [6]:
train_data[0]

{'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.'}

In [7]:
en_nlp = spacy.load("en_core_web_sm")
de_nlp = spacy.load("de_core_news_sm")

In [8]:
string = "What a lovely day it is today!"

[token.text for token in en_nlp.tokenizer(string)]

['What', 'a', 'lovely', 'day', 'it', 'is', 'today', '!']


## `tokenize_example` 函数详解

### 功能描述
这是一个用于处理双语平行语料（英语-德语）的分词预处理函数，主要完成以下转换：
```
原始句子 → 分词 → 大小写处理 → 添加特殊标记
```

### 参数说明
| 参数 | 类型 | 作用 |
|------|------|------|
| `example` | dict | 包含"en"(英文)和"de"(德文)键的字典 |
| `en_nlp` | spaCy模型 | 英语分词器 |
| `de_nlp` | spaCy模型 | 德语分词器 |
| `max_length` | int | 最大分词长度（截断超长句子） |
| `lower` | bool | 是否转为小写 |
| `sos_token` | str | 句子起始标记 |
| `eos_token` | str | 句子结束标记 |

### 设计目的
1. 统一不同语言的预处理流程
2. 为Seq2Seq模型准备标准化的输入格式
3. 控制输入序列的最大长度，防止内存溢出
```

In [9]:
def tokenize_example(example, en_nlp, de_nlp, max_length, lower, sos_token, eos_token):
    en_tokens = [token.text for token in en_nlp.tokenizer(example["en"])][:max_length]
    de_tokens = [token.text for token in de_nlp.tokenizer(example["de"])][:max_length]
    if lower:
        en_tokens = [token.lower() for token in en_tokens]
        de_tokens = [token.lower() for token in de_tokens]
    en_tokens = [sos_token] + en_tokens + [eos_token]
    de_tokens = [sos_token] + de_tokens + [eos_token]
    return {"en_tokens": en_tokens, "de_tokens": de_tokens}

```markdown
## 特殊标记与map函数解析

### <sos>和<eos>标记详解

| 标记    | 全称                  | 作用                                                                 |
|---------|-----------------------|----------------------------------------------------------------------|
| `<sos>` | Start Of Sentence     | 标识句子开始位置，为模型提供序列起始信号                             |
| `<eos>` | End Of Sentence       | 标识句子结束位置，用于控制生成文本长度和标记序列终止                 |

**关键特性**：
1. 在序列到序列(Seq2Seq)模型中：
   - 编码器：通常只需添加`<eos>`
   - 解码器：需要同时添加`<sos>`和`<eos>`
2. 实际实现时会被转换为特殊ID（如`<sos>=1`, `<eos>=2`）

### map函数工作机制

```python
dataset.map(
    tokenize_example,    # 应用的处理函数
    fn_kwargs=fn_kwargs  # 传递给函数的参数字典
)
```

**核心功能**：
1. **批量处理**：自动遍历数据集所有样本应用处理函数
2. **参数传递**：通过`fn_kwargs`将配置参数传递给每个处理函数
3. **惰性执行**：HuggingFace数据集特有的延迟计算机制

**处理流程**：
1. 对每个样本调用`tokenize_example(example, **fn_kwargs)`
2. 自动维护处理后的数据集结构
3. 只有在实际访问数据时才执行计算（内存优化）

**等效实现**：
```python
processed_data = []
for example in train_data:
    processed_data.append(tokenize_example(example, **fn_kwargs))
```

**优势**：
- 代码简洁性：1行代码替代显式循环
- 性能优化：自动并行处理（可通过`num_proc`参数控制）
- 数据一致性：确保所有数据应用相同的预处理逻辑
```

In [10]:
max_length = 1_000
lower = True
sos_token = "<sos>"
eos_token = "<eos>"

fn_kwargs = {
    "en_nlp": en_nlp,
    "de_nlp": de_nlp,
    "max_length": max_length,
    "lower": lower,
    "sos_token": sos_token,
    "eos_token": eos_token,
}

train_data = train_data.map(tokenize_example, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(tokenize_example, fn_kwargs=fn_kwargs)
test_data = test_data.map(tokenize_example, fn_kwargs=fn_kwargs)

Map:   0%|          | 0/1014 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [11]:

train_data[0]

{'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
 'en_tokens': ['<sos>',
  'two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.',
  '<eos>'],
 'de_tokens': ['<sos>',
  'zwei',
  'junge',
  'weiße',
  'männer',
  'sind',
  'im',
  'freien',
  'in',
  'der',
  'nähe',
  'vieler',
  'büsche',
  '.',
  '<eos>']}

In [12]:
min_freq = 2
unk_token = "<unk>"
pad_token = "<pad>"

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token,
]

en_vocab = torchtext.vocab.build_vocab_from_iterator(
    train_data["en_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)

de_vocab = torchtext.vocab.build_vocab_from_iterator(
    train_data["de_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)


In [13]:
en_vocab.get_itos()[:10]

['<unk>', '<pad>', '<sos>', '<eos>', 'a', '.', 'in', 'the', 'on', 'man']

In [14]:
de_vocab.get_itos()[:10]

['<unk>', '<pad>', '<sos>', '<eos>', '.', 'ein', 'einem', 'in', 'eine', ',']

In [15]:
en_vocab["the"]

7

In [16]:
assert en_vocab[unk_token] == de_vocab[unk_token]
assert en_vocab[pad_token] == de_vocab[pad_token]

unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]

In [17]:
en_vocab.set_default_index(unk_index)
de_vocab.set_default_index(unk_index)


In [18]:
tokens = ["i", "love", "watching", "crime", "shows"]
en_vocab.lookup_indices(tokens)

[956, 2169, 173, 0, 821]

```markdown
## 词汇表转换中"crime"变成<unk>的原因解析

### 核心原因
"crime"被转换成`<unk>`是因为该单词没有被包含在`en_vocab`词汇表中，而`set_default_index(unk_index)`设置了默认处理方式。

### 详细解释

1. **词汇表工作原理**：
   - `en_vocab`是一个预定义的词汇表对象
   - 它包含训练集中出现过的单词及其对应索引
   - 通过`set_default_index(unk_index)`设置了未知词的处理方式

2. **转换过程分析**：
   ```python
   # 原始token列表
   tokens = ["i", "love", "watching", "crime", "shows"]
   
   # 转换为索引时
   en_vocab.lookup_indices(tokens) → [956, 2169, 173, 0, 821]
   # "crime"不在词汇表中，返回预设的unk_index(0)
   
   # 索引转回token时
   en_vocab.lookup_tokens([...,0,...]) → [...,'<unk>',...]
   # 索引0对应<unk>标记
   ```

3. **为什么会出现这种情况**：
   - 训练数据量不足，未覆盖所有可能单词
   - "crime"可能是低频词，被词汇表过滤掉了
   - 可能是预处理阶段被错误地移除了
```

In [19]:
en_vocab.lookup_tokens(en_vocab.lookup_indices(tokens))

['i', 'love', 'watching', '<unk>', 'shows']

```markdown
## 代码功能解析

### numericalize_example 函数

**核心作用**：
- 实现文本token到数值ID的转换
- 为神经网络模型准备数值化输入

**处理逻辑**：
1. 接收包含双语token序列的字典输入
2. 分别查询英语和德语的词汇表
3. 将token序列转换为对应的ID序列
4. 返回包含转换结果的字典

### map函数应用

核心作用：
• 批量执行数值化转换

• 保持数据集处理的一致性


实现机制：
1. 通过fn_kwargs传递词汇表参数
2. 自动遍历数据集所有样本
3. 对每个样本应用numericalize_example函数
4. 返回转换后的新数据集

In [20]:
def numericalize_example(example, en_vocab, de_vocab):
    en_ids = en_vocab.lookup_indices(example["en_tokens"])
    de_ids = de_vocab.lookup_indices(example["de_tokens"])
    return {"en_ids": en_ids, "de_ids": de_ids}

In [21]:
fn_kwargs = {"en_vocab": en_vocab, "de_vocab": de_vocab}

train_data = train_data.map(numericalize_example, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(numericalize_example, fn_kwargs=fn_kwargs)
test_data = test_data.map(numericalize_example, fn_kwargs=fn_kwargs)

Map:   0%|          | 0/1014 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [22]:

train_data[0]

{'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
 'en_tokens': ['<sos>',
  'two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.',
  '<eos>'],
 'de_tokens': ['<sos>',
  'zwei',
  'junge',
  'weiße',
  'männer',
  'sind',
  'im',
  'freien',
  'in',
  'der',
  'nähe',
  'vieler',
  'büsche',
  '.',
  '<eos>'],
 'en_ids': [2, 16, 24, 15, 25, 778, 17, 57, 80, 202, 1312, 5, 3],
 'de_ids': [2, 18, 26, 253, 30, 84, 20, 88, 7, 15, 110, 7647, 3171, 4, 3]}

In [23]:
data_type = "torch"
format_columns = ["en_ids", "de_ids"]

train_data = train_data.with_format(
    type=data_type, columns=format_columns, output_all_columns=True
)

valid_data = valid_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

test_data = test_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

In [24]:

train_data[0]

{'en_ids': tensor([   2,   16,   24,   15,   25,  778,   17,   57,   80,  202, 1312,    5,
            3]),
 'de_ids': tensor([   2,   18,   26,  253,   30,   84,   20,   88,    7,   15,  110, 7647,
         3171,    4,    3]),
 'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
 'en_tokens': ['<sos>',
  'two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.',
  '<eos>'],
 'de_tokens': ['<sos>',
  'zwei',
  'junge',
  'weiße',
  'männer',
  'sind',
  'im',
  'freien',
  'in',
  'der',
  'nähe',
  'vieler',
  'büsche',
  '.',
  '<eos>']}

```markdown
## 数据处理函数解析

### `get_collate_fn` 函数

**作用**：
创建用于处理批次数据的整理函数(collate function)，主要实现：
1. 从批次中提取英语和德语的ID序列
2. 对不等长序列进行填充对齐
3. 返回整理后的批次数据

**关键参数**：
- `pad_index`: 用于填充的索引值

**实现细节**：
1. 内部定义`collate_fn`函数处理实际批次数据
2. 使用`nn.utils.rnn.pad_sequence`进行序列填充
3. 返回包含填充后序列的字典

### `get_data_loader` 函数

**作用**：
创建PyTorch数据加载器(DataLoader)，主要功能：
1. 配置数据加载参数
2. 应用自定义的整理函数
3. 控制数据加载行为

**关键参数**：
- `dataset`: 要加载的数据集
- `batch_size`: 批次大小
- `pad_index`: 传递给整理函数的填充索引
- `shuffle`: 是否打乱数据顺序

**实现细节**：
1. 调用`get_collate_fn`获取整理函数
2. 创建PyTorch DataLoader实例
3. 配置批次加载和顺序控制

**组合使用效果**：
这两个函数配合实现了：
- 自动批次处理
- 序列长度对齐
- 高效数据加载
为模型训练提供格式统一的批次数据
```

In [25]:
def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_en_ids = [example["en_ids"] for example in batch]
        batch_de_ids = [example["de_ids"] for example in batch]
        batch_en_ids = nn.utils.rnn.pad_sequence(batch_en_ids, padding_value=pad_index)
        batch_de_ids = nn.utils.rnn.pad_sequence(batch_de_ids, padding_value=pad_index)
        batch = {
            "en_ids": batch_en_ids,
            "de_ids": batch_de_ids,
        }
        return batch

    return collate_fn

In [26]:
def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    return data_loader
get_collate_fn(pad_index)

<function __main__.get_collate_fn.<locals>.collate_fn(batch)>

In [27]:

batch_size = 128

train_data_loader = get_data_loader(train_data, batch_size, pad_index, shuffle=True)
valid_data_loader = get_data_loader(valid_data, batch_size, pad_index)
test_data_loader = get_data_loader(test_data, batch_size, pad_index)


```markdown
## Encoder类解析

### 输入参数
| 参数名 | 类型 | 描述 |
|--------|------|------|
| `input_dim` | int | 输入词汇表大小 |
| `embedding_dim` | int | 词嵌入维度 |
| `hidden_dim` | int | LSTM隐藏层维度 |
| `n_layers` | int | LSTM堆叠层数 |
| `dropout` | float | Dropout概率 |

### 核心组件
1. **词嵌入层** (`nn.Embedding`)
   - 将输入的单词索引映射为密集向量表示
   - 维度转换：`[seq_len, batch_size]` → `[seq_len, batch_size, embedding_dim]`

2. **LSTM层** (`nn.LSTM`)
   - 处理变长序列输入
   - 支持多层堆叠和dropout
   - 自动维护隐藏状态和细胞状态

3. **Dropout层** (`nn.Dropout`)
   - 在训练时随机丢弃部分神经元
   - 防止过拟合，提高模型泛化能力

### forward处理流程
1. **输入嵌入**
   ```python
   embedded = self.dropout(self.embedding(src))
   ```
   - 输入形状：`[src_len, batch_size]`
   - 输出形状：`[src_len, batch_size, embedding_dim]`

2. **LSTM处理**
   ```python
   outputs, (hidden, cell) = self.rnn(embedded)
   ```
   - 自动处理变长序列
   - 返回：
     - `outputs`: 每个时间步的顶层隐藏状态
     - `hidden`: 最后时间步的所有层隐藏状态
     - `cell`: 最后时间步的所有层细胞状态

### 输出说明
| 输出 | 形状 | 描述 |
|------|------|------|
| `outputs` | `[src_len, batch_size, hidden_dim]` | 每个时间步的编码表示 |
| `hidden` | `[n_layers, batch_size, hidden_dim]` | 最终隐藏状态（用于Decoder初始化） |
| `cell` | `[n_layers, batch_size, hidden_dim]` | 最终细胞状态（用于Decoder初始化） |

### 设计特点
1. 支持批处理并行计算
2. 自动处理变长序列
3. 通过dropout实现正则化
4. 输出包含完整的序列信息和最终状态
```

In [28]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # src = [src length, batch size]
        embedded = self.dropout(self.embedding(src))
        # embedded = [src length, batch size, embedding dim]
        outputs, (hidden, cell) = self.rnn(embedded)
        # outputs = [src length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # outputs are always from the top hidden layer
        return hidden, cell

## Decoder类解析

### 核心功能
Decoder实现序列生成任务，将编码后的表示逐步解码为目标语言序列。采用典型的LSTM架构，通过自回归方式逐个生成输出token。

### 输入参数
| 参数 | 类型 | 描述 |
|------|------|------|
| `input` | LongTensor | 当前时间步的输入token索引，形状为`[batch_size]` |
| `hidden` | FloatTensor | 编码器传递的隐藏状态，形状`[n_layers, batch_size, hidden_dim]` |
| `cell` | FloatTensor | 编码器传递的细胞状态，形状`[n_layers, batch_size, hidden_dim]` |

### 组件结构
1. **词嵌入层** (`nn.Embedding`)
   - 将离散token索引映射为连续向量空间
   - 输入维度：`output_dim` (目标词汇表大小)
   - 输出维度：`embedding_dim`

2. **LSTM层** (`nn.LSTM`)
   - 多层LSTM结构，保持与Encoder相同的隐藏维度
   - 使用dropout进行正则化
   - 处理序列生成任务

3. **全连接层** (`nn.Linear`)
   - 将LSTM输出投影到目标词汇表空间
   - 计算每个token的生成概率

### 处理流程
1. **输入预处理**
   ```python
   input = input.unsqueeze(0)  # [batch_size] → [1, batch_size]

In [29]:
class Decoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        # input = [batch size]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # n directions in the decoder will both always be 1, therefore:
        # hidden = [n layers, batch size, hidden dim]
        # context = [n layers, batch size, hidden dim]
        input = input.unsqueeze(0)
        # input = [1, batch size]
        embedded = self.dropout(self.embedding(input))
        # embedded = [1, batch size, embedding dim]
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # output = [seq length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # seq length and n directions will always be 1 in this decoder, therefore:
        # output = [1, batch size, hidden dim]
        # hidden = [n layers, batch size, hidden dim]
        # cell = [n layers, batch size, hidden dim]
        prediction = self.fc_out(output.squeeze(0))
        # prediction = [batch size, output dim]
        return prediction, hidden, cell

```markdown
## Seq2Seq类解析

### 整体架构
Seq2Seq模型由Encoder和Decoder组成，实现端到端的序列转换任务（如机器翻译）。通过Encoder编码源序列，Decoder逐步生成目标序列。

### forward处理流程

1. **初始化阶段**
   ```python
   outputs = torch.zeros(trg_length, batch_size, trg_vocab_size).to(self.device)
   hidden, cell = self.encoder(src)
   input = trg[0, :]  # 初始输入为<sos>标记
   ```
   - 创建输出张量存储每个时间步的预测
   - 通过Encoder获取源序列的编码状态
   - 使用目标序列的首个token(<sos>)初始化Decoder

2. **序列生成循环**
   ```python
   for t in range(1, trg_length):
       output, hidden, cell = self.decoder(input, hidden, cell)
       outputs[t] = output
       top1 = output.argmax(1)  # 获取预测token
       input = trg[t] if teacher_force else top1  # 选择输入来源
   ```
   - 每个时间步：
     1. 通过Decoder生成当前输出和更新状态
     2. 存储预测分布到outputs
     3. 根据teacher forcing策略决定下一步输入

3. **输出结果**
   - 返回形状为`[trg_length, batch_size, trg_vocab_size]`的预测张量
   - 每个位置包含对应时间步的词汇表概率分布

### Teacher Forcing机制

| 机制 | 说明 | 数学表示 |
|------|------|----------|
| **启用时** (概率=teacher_forcing_ratio) | 使用真实目标序列作为Decoder输入 | $input_t = trg_t$ |
| **禁用时** | 使用Decoder上一步的预测结果作为输入 | $input_t = \argmax(output_{t-1})$ |

**工作特点**：
1. 训练时随机切换两种模式（通过`random.random()`实现）
2. 典型设置：初始teacher_forcing_ratio=0.75，随着训练逐步降低
3. 验证/测试时通常设为0（完全使用模型自身预测）

### 设计要点
1. **维度一致性检查**：构造函数验证Encoder/Decoder的hidden_dim和n_layers匹配
2. **设备管理**：显式处理张量设备（通过self.device）
3. **序列生成**：通过循环实现自回归生成，支持变长输出
4. **灵活控制**：teacher_forcing_ratio可动态调整训练行为
```

In [30]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        assert (
            encoder.hidden_dim == decoder.hidden_dim
        ), "Hidden dimensions of encoder and decoder must be equal!"
        assert (
            encoder.n_layers == decoder.n_layers
        ), "Encoder and decoder must have equal number of layers!"

    def forward(self, src, trg, teacher_forcing_ratio):
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        batch_size = trg.shape[1]
        trg_length = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        # tensor to store decoder outputs
        outputs = torch.zeros(trg_length, batch_size, trg_vocab_size).to(self.device)
        # last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # first input to the decoder is the <sos> tokens
        input = trg[0, :]
        # input = [batch size]
        for t in range(1, trg_length):
            # insert input token embedding, previous hidden and previous cell states
            # receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            # output = [batch size, output dim]
            # hidden = [n layers, batch size, hidden dim]
            # cell = [n layers, batch size, hidden dim]
            # place predictions in a tensor holding predictions for each token
            outputs[t] = output
            # decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            # get the highest predicted token from our predictions
            top1 = output.argmax(1)
            # if teacher forcing, use actual next token as next input
            # if not, use predicted token
            input = trg[t] if teacher_force else top1
            # input = [batch size]
        return outputs

In [31]:
input_dim = len(de_vocab)
output_dim = len(en_vocab)
encoder_embedding_dim = 256
decoder_embedding_dim = 256
hidden_dim = 512
n_layers = 2
encoder_dropout = 0.5
decoder_dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 编码器初始化
encoder = Encoder(
    input_dim,
    encoder_embedding_dim,
    hidden_dim,
    n_layers,
    encoder_dropout,
)
# 解码器初始化
decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    hidden_dim,
    n_layers,
    decoder_dropout,
)
# Seq2Seq模型整合
model = Seq2Seq(encoder, decoder, device).to(device)

In [32]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)


model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [33]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 13,898,501 trainable parameters


In [34]:
optimizer = optim.Adam(model.parameters())

In [35]:
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

In [36]:
def train_fn(
    model, data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device
):
    # 设置模型为训练模式
    model.train()
    # 初始化当前epoch的总损失
    epoch_loss = 0
    # 遍历数据加载器中的每个批次
    for i, batch in enumerate(data_loader):
        # 获取源语言序列并移动到指定设备
        src = batch["de_ids"].to(device)
        # 获取目标语言序列并移动到指定设备
        trg = batch["en_ids"].to(device)
        # src的形状为 [src length, batch size]
        # trg的形状为 [trg length, batch size]
        
        # 清空优化器的梯度
        optimizer.zero_grad()
        # 将源序列和目标序列输入模型，获取输出
        output = model(src, trg, teacher_forcing_ratio)
        # output的形状为 [trg length, batch size, trg vocab size]
        
        # 获取目标词汇表的大小
        output_dim = output.shape[-1]
        # 去掉输出的第一个时间步（<sos>），并调整形状以适配损失函数
        output = output[1:].view(-1, output_dim)
        # output的形状为 [(trg length - 1) * batch size, trg vocab size]
        
        # 去掉目标序列的第一个时间步（<sos>），并调整形状以适配损失函数
        trg = trg[1:].view(-1)
        # trg的形状为 [(trg length - 1) * batch size]
        
        # 计算当前批次的损失
        loss = criterion(output, trg)
        # 反向传播计算梯度
        loss.backward()
        # 对梯度进行裁剪，防止梯度爆炸
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        # 更新模型参数
        optimizer.step()
        # 累加当前批次的损失
        epoch_loss += loss.item()
    
    # 返回当前epoch的平均损失
    return epoch_loss / len(data_loader)



In [37]:
def evaluate_fn(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            src = batch["de_ids"].to(device)
            trg = batch["en_ids"].to(device)
            # src = [src length, batch size]
            # trg = [trg length, batch size]
            output = model(src, trg, 0)  # turn off teacher forcing
            # output = [trg length, batch size, trg vocab size]
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            # output = [(trg length - 1) * batch size, trg vocab size]
            trg = trg[1:].view(-1)
            # trg = [(trg length - 1) * batch size]
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [38]:
n_epochs = 1 # 因模型训练对计算资源要求较高，此处只设立了一轮训练。
clip = 1.0
teacher_forcing_ratio = 0.5

best_valid_loss = float("inf")

for epoch in tqdm.tqdm(range(n_epochs)):
    train_loss = train_fn(
        model,
        train_data_loader,
        optimizer,
        criterion,
        clip,
        teacher_forcing_ratio,
        device,
    )
    valid_loss = evaluate_fn(
        model,
        valid_data_loader,
        criterion,
        device,
    )
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "tut1-model.pt")
    print(f"\tTrain Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")
    print(f"\tValid Loss: {valid_loss:7.3f} | Valid PPL: {np.exp(valid_loss):7.3f}")

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [04:59<00:00, 299.49s/it]

	Train Loss:   5.026 | Train PPL: 152.277
	Valid Loss:   4.861 | Valid PPL: 129.150


In [39]:
model.load_state_dict(torch.load("tut1-model.pt"))

<All keys matched successfully>

In [40]:
def translate_sentence(
    sentence,
    model,
    en_nlp,
    de_nlp,
    en_vocab,
    de_vocab,
    lower,
    sos_token,
    eos_token,
    device,
    max_output_length=25,
):
    model.eval()
    with torch.no_grad():
        if isinstance(sentence, str):
            tokens = [token.text for token in de_nlp.tokenizer(sentence)]
        else:
            tokens = [token for token in sentence]
        if lower:
            tokens = [token.lower() for token in tokens]
        tokens = [sos_token] + tokens + [eos_token]
        ids = de_vocab.lookup_indices(tokens)
        tensor = torch.LongTensor(ids).unsqueeze(-1).to(device)
        hidden, cell = model.encoder(tensor)
        inputs = en_vocab.lookup_indices([sos_token])
        for _ in range(max_output_length):
            inputs_tensor = torch.LongTensor([inputs[-1]]).to(device)
            output, hidden, cell = model.decoder(inputs_tensor, hidden, cell)
            predicted_token = output.argmax(-1).item()
            inputs.append(predicted_token)
            if predicted_token == en_vocab[eos_token]:
                break
        tokens = en_vocab.lookup_tokens(inputs)
    return tokens

In [41]:
sentence = test_data[0]["de"]
expected_translation = test_data[0]["en"]

sentence, expected_translation

('Ein Mann mit einem orangefarbenen Hut, der etwas anstarrt.',
 'A man in an orange hat starring at something.')

In [42]:
translation = translate_sentence(
    sentence,
    model,
    en_nlp,
    de_nlp,
    en_vocab,
    de_vocab,
    lower,
    sos_token,
    eos_token,
    device,
)

In [43]:
translation

['<sos>',
 'a',
 'man',
 'in',
 'a',
 'a',
 'shirt',
 'is',
 'a',
 'a',
 'a',
 '.',
 '<eos>']